In [1]:
from google.colab import drive
drive.mount('/content/drive')  #listになかったのでドライブ上にデータ保存し、ドライブをマウント。

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
from itertools import product

In [38]:
# each prediction
train_origin = pd.read_csv("/content/drive/MyDrive/signate/債務不履行/データセット/train.csv")

clf_feat_lst = [
    "lgbm_feat01",
    "logisticregression_feat03",
    "tabnet_feat00",
    "catboost_feat00",
] # "tabnet_feat00","randomforest_feat02",

oof_pred_df_lst = [pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/oof_pred_{clf_feat}.csv") for clf_feat in clf_feat_lst]
test_pred_df_lst = [pd.read_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/test_pred_{clf_feat}.csv") for clf_feat in clf_feat_lst]

In [39]:
# tag for filename
clf_feat_stacking = "_".join(["stacking"] + [cf.split("_")[0] for cf in clf_feat_lst])

In [40]:
clf_feat_stacking

'stacking_lgbm_logisticregression_tabnet_catboost'

train layer2 model (logistic regression)

In [41]:
#  oof_pred_df_lstは、複数のモデルからのOOF予測結果を含むDataFrameのリスト
# train_originは、オリジナルの訓練データを含むDataFrameで、"health"カラムを含む

# ステップ1: 複数のOOF予測DataFrameを横方向に結合
# 各DataFrameにモデル固有のプレフィックスを付ける
train = pd.concat([oof_pred_df.add_prefix(f"model{i}_") for i, oof_pred_df in enumerate(oof_pred_df_lst)], axis=1)

# ステップ2: 予測ターゲットを追加
train['MIS_Status'] = train_origin['MIS_Status']

# ステップ3: 解析に使用するカラムのリストを取得
cols_exp = train.columns.drop(["MIS_Status"])
col_target = "MIS_Status"

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

x = train[cols_exp].to_numpy()
y = train[col_target].to_numpy()

# 学習データ全体でstandard scalerをfitting (testのstackingで利用される想定)
scaler_all = StandardScaler()
scaler_all.fit(x)

# cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)
y_valid_pred_lst = []
idx_valid_lst = []
clf_lst = []

for idx_train, idx_valid in kf.split(x):
    # partitioning
    x_train = x[idx_train, :]
    x_valid = x[idx_valid, :]
    y_train = y[idx_train]
    y_valid = y[idx_valid]

    # normalization
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_valid = scaler.transform(x_valid)

    # training layer2
    clf = LogisticRegression(random_state=0).fit(x_train, y_train)

    # oof
    y_valid_pred = clf.predict_proba(x_valid)
    y_valid_pred_lst.append(y_valid_pred)
    idx_valid_lst.append(idx_valid)
    clf_lst.append(clf)

idx_valid = np.hstack(idx_valid_lst)
y_valid_pred = np.vstack(y_valid_pred_lst)
oof_pred = y_valid_pred[np.argsort(idx_valid)]

# to dataframe
oof_pred_df = pd.DataFrame(oof_pred, columns=[f"MIS_Status_{h}" for h in range(2)])
oof_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/oof_pred_{clf_feat_stacking}.csv", index=False)

In [34]:
from sklearn.metrics import f1_score

# 実際のターゲット値 y_true の取得
y_true = y  # 実際のターゲット値が保持されている変数

# OOF予測値をクラスに変換（確率から最も高いクラスを選択）
y_oof_pred_class = (oof_pred[:, 1] > 0.5).astype(int)

# mean F1スコア（マクロ平均）を計算
mean_f1_score_macro = f1_score(y_true, y_oof_pred_class, average='macro')
print(f'Mean F1 Score (Macro): {mean_f1_score_macro}')

Mean F1 Score (Macro): 0.6475248424552155


calcurate prob of test

In [35]:
def predict_test(x_test, clf_lst):
    y_test_pred_lst = []

    for clf in clf_lst:
        y_test_pred = clf.predict_proba(x_test)
        y_test_pred_lst.append(y_test_pred)

    y_test_pred = np.mean(y_test_pred_lst, axis=0)
    return y_test_pred

In [36]:
# test_pred_df_lstは、複数のモデルからのテスト予測結果を含むDataFrameのリスト
# scaler_allは、訓練データにfitされたStandardScalerインスタンス
# clf_lstは、アンサンブルに使用する分類器のリスト
# clf_feat_stackingは、アンサンブルモデルの特徴量スタッキングの識別子

# 各予測の結合
test = pd.concat([pred_df.add_prefix(f"model{i}_") for i, pred_df in enumerate(test_pred_df_lst)], axis=1)

# 正規化
x_test = scaler_all.transform(test[cols_exp].values)

# アンサンブル予測確率の計算
y_test_pred = predict_test(x_test, clf_lst)

# 結果の記録
test_pred_df = pd.DataFrame(y_test_pred, columns=[f"MIS_Status_{h}" for h in range(2)])
test_pred_df.to_csv(f"/content/drive/MyDrive/signate/債務不履行/model_feat/test_pred_{clf_feat_stacking}.csv", index=False)